# ライブラリのインストール

In [20]:
!pip install optuna -q
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
import time
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# データ読み込み

In [21]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/github/利益予測データベース.xlsx'
df = pd.read_excel(file_path)
dt = pd.read_excel(file_path)
print(dt.head())
data = df.values

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
           sample  group      ni_t     ni_t1     ni_t2     ni_t3     ni_t4  \
0  201003N0000001      1  0.053142  0.077657  0.073253  0.097867  0.098210   
1  200903N0000001      1  0.083942  0.079182  0.105787  0.106157  0.073416   
2  200803N0000001      1  0.052085  0.069586  0.069830  0.048293  0.043422   
3  200703N0000001      1  0.060600  0.060812  0.042056  0.037814  0.033996   
4  200603N0000001      1  0.080197  0.055463  0.049869  0.044834  0.040998   

      ni_t5   cy_e_t1  cy_sale_t1  ...  cy_iva_c_t1  cy_intan_c_t1  \
0  0.067920  0.077657    7.220336  ...    -0.025984       0.009102   
1  0.066011  0.079182    7.815907  ...    -0.073469       0.002539   
2  0.039038  0.069586    5.465569  ...    -0.083399       0.000139   
3  0.031088  0.060812    4.632824  ...     0.105868       0.005575   
4  0.037042  0.055463    6.099235  ...    -0.012547 

# 訓練データとテストデータに分割

In [22]:
# data[:, 1] == 1 → 訓練データ（例：ラベル1が訓練データを意味する場合）
# data[:, 1] == 2 → テストデータ（例：ラベル2がテストデータを意味する場合）
x_train = data[:, 8:66][data[:, 1] == 1]
y_train = data[:, 2][data[:, 1] == 1]
x_test = data[:, 8:66][data[:, 1] == 2]
y_test = data[:, 2][data[:, 1] == 2]

# Optuna 目的関数定義

| 代码                          | 作用（日语）       | 用法例子           |
| --------------------------- | ------------ | -------------- |
| `trial.suggest_categorical` | 候補リストから選ぶ    | `[64, 128]` など |
| `trial.suggest_int`         | 整数を範囲から選ぶ    | `2, 4`         |
| `trial.suggest_float`       | 実数を範囲から選ぶ    | `0.0, 0.2`     |
| `trial.suggest_float`  | 対数スケールの実数を選ぶ | `1e-4, 1e-1, log=True`   |

In [25]:
def objective(trial):
    # Optunaでハイパーパラメータ自動探索
    params = {
        'objective': 'regression_l1',  # MAE回帰
        'metric': 'mae',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.05),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1
    }
    model = lgb.LGBMRegressor(**params) # Dictionary Unpacking
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    mae = mean_absolute_error(y_test, y_pred)
    return mae  # MAEが小さいほど良い

 lgb.LGBMRegressor(**params)
    # `**` を付けることで、params辞書が展開（アンパック）され、
    # 中のキーと値がそれぞれキーワード引数(ひきすう)としてモデルに渡されます。
    # つまり、以下のようにコードを書いたのと同じ意味になります。
    # lgb.LGBMRegressor(objective='regression_l1',
    #                   metric='mae',
    #                   n_estimators=100,
    #                   learning_rate=0.05,
    #                   ...)
    model = lgb.LGBMRegressor(**params)

| 参数                  | 中文说明               | 日语说明                | 常见调节建议                   |
| ------------------- | ------------------ | ------------------- | ------------------------ |
| objective           | 回归/损失函数（MAE，绝对值损失） | 目的関数/損失関数（MAE，絶対誤差） | 固定为回归问题时用 regression\_l1 |
| metric              | 评估指标（平均绝对误差）       | 評価指標（MAE）           | 固定                       |
| n\_estimators       | 树的棵数               | 決定木の本数              | 100-1000，树越多训练越慢         |
| learning\_rate      | 学习率                | 学習率                 | 0.005-0.1，越小越稳健          |
| max\_depth          | 最大树深               | 木の深さの最大値            | 3-10，越大模型越复杂             |
| num\_leaves         | 叶子数                | 葉の数                 | 20-150，越大表达力强，易过拟合       |
| min\_child\_samples | 叶节点最小样本数           | 葉の最小データ数            | 防止叶节点太小，5-50             |
| subsample           | 行采样比例              | データサンプリング率          | 0.6-1.0，防止过拟合            |
| colsample\_bytree   | 列采样比例              | 特徴量サンプリング率          | 0.6-1.0，防止过拟合            |
| reg\_alpha          | L1正则               | L1正則化項              | 大一些会让模型稀疏化，减少过拟合         |
| reg\_lambda         | L2正则               | L2正則化項              | 控制参数大小，减少过拟合             |
| random\_state       | 随机种子               | 乱数シード               | 固定用于复现                   |
| n\_jobs             | 并行数                | 並列計算数               | -1使用全部CPU                |
| verbose             | 日志详细度              | ログの出力レベル            | -1 静默                    |


# Optuna実行

In [26]:
import plotly.io as pio
# Colab環境でプロットを正しく表示するための設定です
pio.renderers.default = 'colab'

# ステップ1: studyオブジェクトを作成します。
# 私たちの目標はMAE（平均絶対誤差）を最小にすることなので、direction='minimize'と設定します。
study = optuna.create_study(direction='minimize')

# ステップ2: 最適化を実行します。
# objective関数を30回試行(n_trials=50)して、最適なハイパーパラメータを探します。
# この処理には数分かかることがあります。
print("optunaによるハイパーパラメータ探索を開始します...")
study.optimize(objective, n_trials=50)


# 探索完了後に結果を出力します。
print("\n探索が完了しました。")
print(f"最良スコア (検証MAE): {study.best_value}")
print("最良のハイパーパラメータ:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# ステップ3: 結果を可視化します。
from optuna.visualization import plot_optimization_history, plot_param_importances


print("\n--------------------------- 可視化結果 ------------------------------")

# 探索履歴のプロット
# 各試行でMAEがどのように改善されていったかを確認できます。
fig1 = plot_optimization_history(study)
fig1.show()


# ハイパーパラメータの重要度のプロット
# どのパラメータが最終的なスコアに最も影響を与えたかを確認できます。
fig2 = plot_param_importances(study)
fig2.show()


[I 2025-06-29 03:44:20,412] A new study created in memory with name: no-name-5a3f918f-f94d-46d4-8375-ac694f9e67d2


optunaによるハイパーパラメータ探索を開始します...


[I 2025-06-29 03:44:22,350] Trial 0 finished with value: 0.04923020373506031 and parameters: {'n_estimators': 350, 'learning_rate': 0.060927812783200634, 'max_depth': 3, 'num_leaves': 64, 'min_child_samples': 15, 'subsample': 0.7, 'colsample_bytree': 0.65, 'reg_alpha': 0.13619895529873635, 'reg_lambda': 0.08509605593111053}. Best is trial 0 with value: 0.04923020373506031.
[I 2025-06-29 03:44:28,593] Trial 1 finished with value: 0.049251580940025096 and parameters: {'n_estimators': 650, 'learning_rate': 0.01927914196071351, 'max_depth': 3, 'num_leaves': 93, 'min_child_samples': 5, 'subsample': 0.9, 'colsample_bytree': 0.85, 'reg_alpha': 5.127526277936975e-06, 'reg_lambda': 0.6386538375031565}. Best is trial 0 with value: 0.04923020373506031.
[I 2025-06-29 03:44:34,375] Trial 2 finished with value: 0.04991863103552537 and parameters: {'n_estimators': 350, 'learning_rate': 0.08780135461063414, 'max_depth': 7, 'num_leaves': 72, 'min_child_samples': 31, 'subsample': 1.0, 'colsample_bytree'


探索が完了しました。
最良スコア (検証MAE): 0.048935889626871926
最良のハイパーパラメータ:
  n_estimators: 450
  learning_rate: 0.057586022732126506
  max_depth: 3
  num_leaves: 113
  min_child_samples: 45
  subsample: 0.85
  colsample_bytree: 0.9
  reg_alpha: 5.795867476518175e-06
  reg_lambda: 0.0724180494365362

--------------------------- 可視化結果 ------------------------------


# 最良パラメータで再学習＆評価

In [27]:
print("--- ステップ4: 最終モデルの訓練と評価を開始します ---")

# 1. Optunaで見つけた最良のハイパーパラメータを取得します
best_params = study.best_params  # ディクショナリ
print("\n取得した最良のハイパーパラメータ:")
print(best_params)

# 2. LightGBMの固定パラメータと合体（metricやobjective、random_state等は明示的に追加！）
final_model_params = {
    'objective': 'regression_l1',
    'metric': 'mae',
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
}
final_model_params.update(best_params)  # best_paramsの内容で上書き

# 3. モデル再学習
final_model = lgb.LGBMRegressor(**final_model_params)
final_model.fit(x_train,y_train)

# 4. テストデータで予測＆評価
final_pred = final_model.predict(x_test)
final_mae = mean_absolute_error(y_test, final_pred)

print("\n--- 再学習したモデルの評価結果 ---")
print(f"最終MAE: {final_mae:.6f}")

--- ステップ4: 最終モデルの訓練と評価を開始します ---

取得した最良のハイパーパラメータ:
{'n_estimators': 450, 'learning_rate': 0.057586022732126506, 'max_depth': 3, 'num_leaves': 113, 'min_child_samples': 45, 'subsample': 0.85, 'colsample_bytree': 0.9, 'reg_alpha': 5.795867476518175e-06, 'reg_lambda': 0.0724180494365362}

--- 再学習したモデルの評価結果 ---
最終MAE: 0.048936
